In [1]:
import numpy as np
import pandas as pd
import sqlite3
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve,auc
from nltk.stem.porter import PorterStemmer


In [2]:
con = sqlite3.connect('database.sqlite') 
filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3 LIMIT 5000""", con)

In [3]:
def partition(x):
    if x < 3:
        return "negative"
    return 'positive'
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative
print("Number of data points in our data", filtered_data.shape)
filtered_data.head(3)

Number of data points in our data (5000, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


In [4]:
display= pd.read_sql_query(""" 
select *from
reviews where score!=3 and UserId="AR5J8UI46CURR"
order by ProductID
""",con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [5]:
sorted_data=filtered_data.sort_values('ProductId',axis=0,ascending=True)

In [6]:
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"},keep='first',inplace=False )
final.shape

(4986, 10)

In [7]:
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

99.72

In [8]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [9]:
final.shape

(4986, 10)

In [10]:
final['Score'].value_counts()

positive    4178
negative     808
Name: Score, dtype: int64

# Bag Of Words

In [11]:
count_vect=CountVectorizer()
final_counts=count_vect.fit_transform(final['Text'].values)# use DataFrame.
# values attribute to return the numpy representation of the given DataFrame.

In [12]:
type(final_counts)

scipy.sparse.csr.csr_matrix

In [13]:
final_counts.shape

(4986, 13510)

In [14]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shubh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
import re # regular expressions
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

stop=set(stopwords.words('english')) #set of stopwords
sno=nltk.stem.SnowballStemmer('english') # intialising snowball stemmer

def cleanhtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext

def cleanpunc(sentence):
    cleaned= re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned= re.sub(r',[.|,|)|(|\|/]',r' ',cleaned)
    return cleaned

print(stop)
print ('*******************************')
print(sno.stem('tasty'))

{'or', "weren't", 'before', 'all', 'isn', 'this', 'such', 'yourself', 'then', 'theirs', 'those', 'yourselves', "wasn't", 'ain', 'does', 's', "couldn't", "isn't", 've', 'with', "doesn't", "it's", 'against', "you'll", 'no', 'd', 'that', 'her', 'when', 'won', 'doing', 'during', "mustn't", "that'll", "you're", 'an', 'your', 'they', 'myself', 'because', 'if', 'herself', 'didn', 'ours', 'in', 'most', 'mustn', 'at', "mightn't", 'through', 'aren', 'any', 'than', 'above', 'for', 'of', 'here', 'same', 'should', 'over', 'our', 'other', 'their', 'been', 'itself', 'shan', 'his', 'so', 'be', 'my', 'own', 'not', 'mightn', 'under', 'can', 'being', 'you', "hadn't", 'hasn', "haven't", 'a', 'them', 'o', 're', 'shouldn', 'which', 'further', 'the', 'few', 'its', 'who', 'some', 'but', 'until', 'both', 'ourselves', 'couldn', 'she', 'needn', 'had', 'i', 'to', 'did', 'me', 'were', 'these', "you'd", 'we', 'about', 'from', 'there', 'ma', 'after', 'just', 'out', 'do', 'once', 'on', 'hadn', "wouldn't", "didn't", '

In [16]:
i=0
str1=' '
final_string=[]
all_positive_words=[]
all_negative_words=[]
s=''
for sent in final['Text'].values:
    filtered_sentence=[]
    sent= cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if(final['Score'].values)[i] == 'positive':
                        all_positive_words.append(s)
                    if(final['Score'].values)[i] == 'negative':
                        all_negative_words.append(s)
                else:
                    continue
            else:
                continue
                
    str1= b" " .join(filtered_sentence)  
    
    final_string.append(str1)
    i+=1

In [17]:
final['CleanedText']=final_string

In [18]:
final.head(10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
2546,2774,B00002NCJC,A196AJHU9EASJN,Alex Chaffee,0,0,positive,1282953600,thirty bucks?,Why is this $[...] when the same product is av...,b'product avail victor trap cours total fli pr...
2547,2775,B00002NCJC,A13RRPGE79XFFH,reader48,0,0,positive,1281052800,Flies Begone,We have used the Victor fly bait for 3 seasons...,b'use victor fli bait cant beat great product'
1145,1244,B00002Z754,A3B8RCEI0FXFI6,B G Chase,10,10,positive,962236800,WOW Make your own 'slickers' !,I just received my shipment and could hardly w...,b'receiv shipment could hard wait tri love cal...
1146,1245,B00002Z754,A29Z5PI9BW2PU3,Robbie,7,7,positive,961718400,Great Product,This was a really good idea and the final prod...,b'realli good idea final product use decal car...
2942,3204,B000084DVR,A1UGDJP1ZJWVPF,"T. Moore ""thoughtful reader""",1,1,positive,1177977600,Good stuff!,I'm glad my 45lb cocker/standard poodle puppy ...,b'glad poodl puppi love stuff trust brand supe...
2941,3203,B000084DVR,A3DKGXWUEP1AI2,"Glenna E. Bauer ""Puppy Mum""",3,3,positive,1163030400,Premium Quality Dog Food!!!,We have been using this food for about 6 month...,b'use food month find excel fact two dog coton...
1071,1161,B000084E1U,A3DH85EYHW4AQH,Eric Hochman,1,1,positive,1140739200,Cats love it!,I have nine cats and they are crazy about thes...,b'nine cat crazi kibbl last thing want cat foo...
2187,2380,B0000CGFV4,A3LSUKN4IFS6VD,Internet Diva,0,0,positive,1312416000,"Nice, Big Pieces & Big Almond Flavor",These were shipped out the day after I ordered...,b'ship day order arriv within live opposit coa...
4695,5097,B0000D94SZ,A2R91PG1XPNO0B,"Kotton Kandee ""not so secretshopper""",4,4,positive,1184198400,"A Summer Treat Fat Free, Guilt Free",This mix is probably not something you would w...,b'mix probabl someth would want use new enough...
2068,2256,B0000DC5IY,AGYZZ3QXV9S8,"Dr. Glenn B. Gero ""NJnaturaldoc""",3,4,negative,1289865600,Don't buy this product unless you are looking ...,The description of this product is disceptive....,b'descript product product repres noth shred c...


In [19]:
conn= sqlite3.connect('final.sqlite')
c=conn.cursor()
conn.text_factory = str
final.to_sql('Reviews', conn, schema=None, if_exists='replace')

In [20]:
freq_pos= nltk.FreqDist(all_positive_words)
freq_neg=nltk.FreqDist(all_negative_words)
print("most common pos words: ",freq_pos.most_common(20))
print("Most common neg words: ",freq_neg.most_common(20))


most common pos words:  [(b'like', 1728), (b'love', 1433), (b'tast', 1324), (b'great', 1227), (b'good', 1196), (b'use', 1188), (b'flavor', 1072), (b'one', 1047), (b'tri', 1023), (b'make', 939), (b'product', 839), (b'get', 810), (b'coffe', 747), (b'chip', 716), (b'food', 697), (b'would', 691), (b'buy', 687), (b'realli', 681), (b'eat', 647), (b'best', 637)]
Most common neg words:  [(b'like', 429), (b'tast', 354), (b'product', 279), (b'tri', 271), (b'one', 248), (b'would', 242), (b'use', 217), (b'flavor', 184), (b'buy', 183), (b'get', 174), (b'food', 171), (b'order', 171), (b'even', 165), (b'make', 161), (b'good', 158), (b'dont', 147), (b'eat', 143), (b'chip', 138), (b'bag', 134), (b'much', 130)]


In [21]:
count_vect=CountVectorizer(ngram_range=(1,2))
final_counts=count_vect.fit_transform(final['Text'].values)

In [22]:
final_counts.shape # includes unigram and bigram

(4986, 148211)

In [23]:
tf_idf_vect=TfidfVectorizer(ngram_range=(1,2))
final_tf_idf=tf_idf_vect.fit_transform(final['Text'].values)

In [24]:
final_tf_idf.shape

(4986, 148211)

In [25]:
features=tf_idf_vect.get_feature_names()
len(features)
print(features[3:10])

['00 at', '00 bag', '00 because', '00 box', '00 br', '00 buy', '00 canned']


C:\Users\shubh\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [26]:
print(final_tf_idf[3,:].toarray()[0])

[0. 0. 0. ... 0. 0. 0.]


In [27]:
 #source: https://buhrmann.github.io/tfidf-analysis.html
def top_tfidf_feats(row,features, top_n=25):
    '''Get top n  tfidf values in row and return them with their corresponding'''
    topn_ids= np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i],row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.column = ['feature', 'tfidf']
    return df

top_tfidf = top_tfidf_feats(final_tf_idf[1,:].toarray()[0],features,25)

C:\Users\shubh\AppData\Local\Temp/ipykernel_22632/2305095445.py:7: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.column = ['feature', 'tfidf']


In [28]:
top_tfidf

,0,1
0,fly bait,0.274736
1,seasons can,0.274736
2,for seasons,0.274736
3,victor,0.262108
4,bait for,0.262108
5,the victor,0.262108
6,victor fly,0.262108
7,fly,0.246199
8,seasons,0.246199
9,bait,0.240521


In [29]:
!pip install gensim

In [30]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

In [31]:
import gensim 
i=0
list_of_sent=[]
for sent in final['Text'].values:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue
    list_of_sent.append(filtered_sentence)
    

In [32]:
print(final['Text'].values[0])
print("**********************************************")
print(list_of_sent[0])

Why is this $[...] when the same product is available for $[...] here?<br />http://www.amazon.com/VICTOR-FLY-MAGNET-BAIT-REFILL/dp/B00004RBDY<br /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.
**********************************************
['why', 'is', 'this', 'when', 'the', 'same', 'product', 'is', 'available', 'for', 'here', 'the', 'victor', 'and', 'traps', 'are', 'of', 'course', 'total', 'fly', 'pretty', 'but', 'only', 'right']


In [33]:
w2v_model=gensim.models.Word2Vec(list_of_sent, min_count=3,workers=4)

In [34]:
words=list(w2v_model.wv.vocab)
print(len(words))

AttributeError: The vocab attribute was removed from KeyedVector in Gensim 4.0.0.
Use KeyedVector's .key_to_index dict, .index_to_key list, and methods .get_vecattr(key, attr) and .set_vecattr(key, attr, new_val) instead.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4